In [1]:
import sys
sys.path.append('../')
import util.PrepareData as pdata
import util.data_processing_tool as dpt

from datetime import date
from netCDF4 import Dataset
import numpy as np
import netCDF4 as nc
from datetime import timedelta, date, datetime

import util.constant_param as consparam


# station_5={
#      'WAGGA WAGGA AMO': [-35.1583, 147.4575],
#      'MENINGIE': [-35.6902, 139.3375],
#      'SURAT': [-27.1591, 149.0702],
#      'MILDURA AIRPORT': [-34.2358, 142.0867],
#      'ORANGE AGRICULTURAL INSTITUTE': [-33.3211, 149.0828],
# }

# station_5_idx={
# #     坐标
#     'WAGGA WAGGA AMO': (98, 176),
#      'MENINGIE': (97, 167),
#      'SURAT': (113, 178),
#      'MILDURA AIRPORT': (100, 170),
#      'ORANGE AGRICULTURAL INSTITUTE': (102, 178)}

# root_dir='../../data/'


E:\Users\Weifa\Anaconda3\envs\py37\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
E:\Users\Weifa\Anaconda3\envs\py37\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
E:\Users\Weifa\Anaconda3\envs\py37\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
E:\Users\Weifa\Anaconda3\envs\py37\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
start_date=date(1990, 1, 2)
end_date=date(2012,12,25)
dates=[start_date + timedelta(x) for x in range((end_date - start_date).days + 1)]
file_BARRA_dir="../../Data/barra_aus/"

def get_barra_cali_data():

    for i in dates:
        data_high=dpt.read_barra_data_fc(file_BARRA_dir,i)
        data_50=[]
        station_name=[]
        for station in consparam.station_50_index_for_size_of_hr_sr_station_code.keys():
#             print(station)
            idx_i=consparam.station_50_index_for_size_of_hr_sr_station_code[station][0]
            idx_j=consparam.station_50_index_for_size_of_hr_sr_station_code[station][1]
            data_50.append(data_high[idx_i][idx_j])
            station_name.append(station)
#             data_50.append(data_high[station[0]][station[1]])

        data_50=np.array(data_50)
        station_name=np.array(station_name)
        dpt.mkdir('../../Data/barra_aus/50_station/')
        f_w = nc.Dataset('../../Data/barra_aus/50_station/'+i.strftime("%Y%m%d")+'.nc','w',format = 'NETCDF4')
        f_w.createDimension('station',50)

        f_w.createVariable('station',np.int32,('station'))


        f_w.variables['station'][:] = station_name
#         f_w.variables['lon'][:] = label["lon"].data

        f_w.createVariable( 'barra', np.float32, ('station'))
        f_w.variables['barra'][:] = data_50

        f_w.close()

#     print(dpt.read_barra_data_fc("../data/barra_aus/",demdo_date)[78,315])

#  'WAGGA WAGGA AMO': [78, 315],
get_barra_cali_data()

In [43]:
nc.Dataset('../../Data/barra_aus/50_station/19900102.nc')

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): station(50)
    variables(dimensions): int32 station(station), float32 barra(station)
    groups: 

In [ ]:
import sys
sys.path.append('../')
import util.PrepareData as pdata
import util.data_processing_tool as dpt

from datetime import date
from netCDF4 import Dataset
import numpy as np
import netCDF4 as nc
from datetime import timedelta, date, datetime

import util.constant_param as consparam
start_date=date(1990, 1, 2)
end_date=date(2012,12,25)
dates=[start_date + timedelta(x) for x in range((end_date - start_date).days + 1)]

#50 station
def get_access_pr_data():# 顺便保存 BI数据
    import netCDF4 as nc
#     ensemble=['e01','e02']
    sys = platform.system()
    init_date=date(1970, 1, 1)
    start_date=date(1990, 1, 1)
    end_date=date(2012,12,25) #if 929 is true we should substract 1 day  
    if sys == "Windows":
        init_date=date(1970, 1, 1)
        start_date=date(1990, 1, 1)
        end_date=date(2012,12,25) #if 929 is true we should substract 1 day   
        file_ACCESS_dir="H:/climate/access-s1/" 
        file_BARRA_dir="D:/dataset/accum_prcp/"
    #         args.file_ACCESS_dir="E:/climate/access-s1/"
    #         args.file_BARRA_dir="C:/Users/JIA059/barra/"
        file_DEM_dir="../DEM/"
    else:
        file_ACCESS_dir_pr="/g/data/ub7/access-s1/hc/raw_model/atmos/pr/daily/"
        file_ACCESS_dir="/g/data/ub7/access-s1/hc/raw_model/atmos/"
        # training_name="temp01"
        file_BARRA_dir="/g/data/ma05/BARRA_R/v1/forecast/spec/accum_prcp/"

        
    nine2nine=True
    date_minus_one=1
    leading_time_we_use=7
    ensemble=['e01','e02','e03','e04','e05','e06','e07','e08','e09','e10','e11']

    var_name="pr"
    dates=[start_date + timedelta(x) for x in range((end_date - start_date).days + 1)]
    file_list=get_filename_with_time_order(file_ACCESS_dir+var_name+'/daily/',ensemble,dates,var_name)
    time_leading=217

    lat_name="lat"
    lon_name="lon"

#     print(file_list)
    for i in file_list:
#         print(i)
        data = Dataset(i[0], 'r')
        var = data[var_name][:,82:144,134:188]*86400
        lat = data[lat_name][:][82:144]
        lon = data[lon_name][:][134:188]
#         print(var.shape)
        data.close()
    #         lr=dpt.read_access_data(i,idx=time_leading).data[82:144,134:188]*86400
        result=np.zeros((217,316,376))
        for idx,j in enumerate(var):
            result[idx]=dpt.interp_tensor_2d( dpt.interp_tensor_2d(j,(79,94) ),(316, 376))

            data_50=[]
            station_name=[]
            for station in consparam.station_50_index_for_size_of_hr_sr_station_code.keys():
                idx_i=consparam.station_50_index_for_size_of_hr_sr_station_code[station][0]
                idx_j=consparam.station_50_index_for_size_of_hr_sr_station_code[station][1]
                data_50.append(result[:,idx_i,idx_j])
                station_name.append(station)
=

        data_50=np.array(data_50)
        station_name=np.array(station_name)
        
        
        if not os.path.exists('../Data/'+var_name+'/daily_BI_50station/'+i[1]):
            os.makedirs('../Data/'+var_name+'/daily_BI_50station/'+i[1])
        
        f_w = nc.Dataset('../Data/'+var_name+'/daily_BI_50station/'+i[1]+"/da_"+var_name+"_"+i[2].strftime("%Y%m%d")+"_"+i[1]+'.nc','w',format = 'NETCDF4')

        f_w.createDimension('time',time_leading)
        f_w.createDimension('station',50)

        f_w.createVariable('station',np.int32,('station'))
        f_w.createVariable('time',np.int,('time'))



        f_w.variables['station'][:] = station_name
        f_w.variables['time'][:] =  np.array(range(1,218))


        f_w.createVariable( 'pr', np.float32, ('station','time'))
        f_w.variables['pr'][:] = data_50

        f_w.close()
            

    
            

    
            
if __name__=='__main__':
    main()

In [41]:
np.array(range(1,218))


array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 18

In [ ]:
import sys
sys.path.append('../')
import util.PrepareData as pdata
import util.data_processing_tool as dpt

from datetime import date
from netCDF4 import Dataset
import numpy as np
import netCDF4 as nc
from datetime import timedelta, date, datetime
import platform 
import util.constant_param as consparam
#50 station
import os


def get_filename_with_no_time_order(rootdir):
    '''get filename first and generate label '''
    _files = []
    list = os.listdir(rootdir) #列出文件夹下所有的目录与文件
    for i in range(0,len(list)):
        path = os.path.join(rootdir,list[i],)
        if os.path.isdir(path):
            _files.extend(get_filename_with_no_time_order(path))
        if os.path.isfile(path):
            if path[-3:]==".nc":
                _files.append(path)
    return _files


def get_filename_with_time_order(rootdir,ensemble,dates,var_name):
    '''get filename first and generate label ,one different w'''
    _files = []
    for en in ensemble:
        for date in dates:
            access_path=rootdir+en+"/"+"daq5_"+var_name+"_"+date.strftime("%Y%m%d")+"_"+en+".nc"
#             print(access_path)
            if os.path.exists(access_path):
#                 print(access_path)
                path=[access_path]
                path.append(en)
                path.append(date)
                _files.append(path)

#最后去掉第一行，然后shuffle
#     if nine2nine and date_minus_one==1:
#         del _files[0]
    return _files




def get_access_cali_data():# 顺便保存 BI数据
    import netCDF4 as nc
#     ensemble=['e01','e02']
    sys = platform.system()
    init_date=date(1970, 1, 1)
    start_date=date(1990, 1, 1)
    end_date=date(2012,12,25) #if 929 is true we should substract 1 day  
    if sys == "Windows":
        init_date=date(1970, 1, 1)
        start_date=date(1990, 1, 1)
        end_date=date(2012,12,25) #if 929 is true we should substract 1 day   
        file_ACCESS_dir="H:/climate/access-s1/" 
        file_BARRA_dir="D:/dataset/accum_prcp/"
    #         args.file_ACCESS_dir="E:/climate/access-s1/"
    #         args.file_BARRA_dir="C:/Users/JIA059/barra/"
        file_DEM_dir="../DEM/"
    else:
        file_ACCESS_dir_pr="/g/data/ub7/access-s1/hc/raw_model/atmos/pr/daily/"
        file_ACCESS_dir = '/g/data/ub7/access-s1/hc/calibrated_5km_v3/atmos/pr/'
        # training_name="temp01"
        file_BARRA_dir="/g/data/ma05/BARRA_R/v1/forecast/spec/accum_prcp/"


    ensemble=['e01','e02','e03','e04','e05','e06','e07','e08','e09','e10','e11']

    var_name="pr"
    dates=[start_date + timedelta(x) for x in range((end_date - start_date).days + 1)]
    file_list=get_filename_with_time_order(file_ACCESS_dir+var_name+'/daily/',ensemble,dates,var_name)
    time_leading=217

    lat_name="lat"
    lon_name="lon"

#     print(file_list)
    for i in file_list:
#         print(i)
        data = Dataset(i[0], 'r')
        var = data[var_name][:]
        lat = data[lat_name][:]
        lon = data[lon_name][:]
#         print(var.shape)
        data.close()

            
        data_50=[]
        station_name=[]
        for station in consparam.station_50_index_for_size_of_calibration_station_code.keys():
            idx_i=consparam.station_50_index_for_size_of_calibration_station_code[station][0]
            idx_j=consparam.station_50_index_for_size_of_calibration_station_code[station][1]
            data_50.append(var[:,idx_i,idx_j])
            station_name.append(station)

        data_50=np.array(data_50)
        station_name=np.array(station_name)
        
        
        if not os.path.exists('../Data/'+var_name+'/daily_BI_50station/'+i[1]):
            os.makedirs('../Data/'+var_name+'/calibrated_5km_v3_50station/'+i[1])
        
        f_w = nc.Dataset('../Data/'+var_name+'/calibrated_5km_v3_50station/'+i[1]+"/daq5_"+var_name+"_"+i[2].strftime("%Y%m%d")+"_"+i[1]+'.nc','w',format = 'NETCDF4')

        f_w.createDimension('time',time_leading)
        f_w.createDimension('station',50)

        f_w.createVariable('station',np.int32,('station'))
        f_w.createVariable('time',np.int,('time'))



        f_w.variables['station'][:] = station_name
        f_w.variables['time'][:] =  np.array(range(1,218))


        f_w.createVariable( 'pr', np.float32, ('station','time'))
        f_w.variables['pr'][:] = data_50
        print(data_50.shape)
        f_w.close()
            

    
            

    
            
if __name__=='__main__':
    get_access_cali_data()

In [52]:
import sys
sys.path.append('../')
import util.PrepareData as pdata
import util.data_processing_tool as dpt

from datetime import date
from netCDF4 import Dataset
import numpy as np
import netCDF4 as nc
from datetime import timedelta, date, datetime

import torch
from torchvision import datasets, models, transforms

from PIL import Image
import util.constant_param as consparam
start_date=date(1990, 1, 2)
end_date=date(2012,12,25)
dates=[start_date + timedelta(x) for x in range((end_date - start_date).days + 1)]

#50 station

def get_filename_with_no_time_order(rootdir):
    '''get filename first and generate label '''
    _files = []
    list = os.listdir(rootdir) #列出文件夹下所有的目录与文件
    for i in range(0,len(list)):
        path = os.path.join(rootdir,list[i],)
        if os.path.isdir(path):
            _files.extend(get_filename_with_no_time_order(path))
        if os.path.isfile(path):
            if path[-3:]==".nc":
                _files.append(path)
    return _files


def get_filename_with_time_order(rootdir,ensemble,dates,var_name):
    '''get filename first and generate label ,one different w'''
    _files = []
    for en in ensemble:
        for date in dates:
            access_path=rootdir+en+"/"+"da_"+var_name+"_"+date.strftime("%Y%m%d")+"_"+en+".nc"
#             print(access_path)
            if os.path.exists(access_path):
#                 print(access_path)
                path=[access_path]
                path.append(en)
                path.append(date)
                _files.append(path)

#最后去掉第一行，然后shuffle
#     if nine2nine and date_minus_one==1:
#         del _files[0]
    return _files



def get_access_SR_data():# 顺便保存 BI数据
    
    
    lr_transforms = transforms.Compose([
    transforms.Resize((316, 376)),
#     transforms.RandomResizedCrop(IMG_SIZE),
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomRotation(30),
    transforms.ToTensor()
#     transforms.Normalize(IMG_MEAN, IMG_STD)
    ])
    
    
    import netCDF4 as nc
#     ensemble=['e01','e02']
    sys = platform.system()
    init_date=date(1970, 1, 1)
    start_date=date(1990, 1, 1)
    end_date=date(2012,12,25) #if 929 is true we should substract 1 day  
    if sys == "Windows":
        init_date=date(1970, 1, 1)
        start_date=date(1990, 1, 1)
        end_date=date(2012,12,25) #if 929 is true we should substract 1 day   
        file_ACCESS_dir="H:/climate/access-s1/" 
        file_BARRA_dir="D:/dataset/accum_prcp/"
    #         args.file_ACCESS_dir="E:/climate/access-s1/"
    #         args.file_BARRA_dir="C:/Users/JIA059/barra/"
        file_DEM_dir="../DEM/"
    else:
        file_ACCESS_dir_pr="/g/data/ub7/access-s1/hc/raw_model/atmos/pr/daily/"
        file_ACCESS_dir="/g/data/ub7/access-s1/hc/raw_model/atmos/"
        # training_name="temp01"
        file_BARRA_dir="/g/data/ma05/BARRA_R/v1/forecast/spec/accum_prcp/"

        
    ensemble=['e01','e02','e03','e04','e05','e06','e07','e08','e09','e10','e11']

    var_name="pr"
    dates=[start_date + timedelta(x) for x in range((end_date - start_date).days + 1)]
    file_list=get_filename_with_time_order(file_ACCESS_dir+var_name+'/daily/',ensemble,dates,var_name)
    time_leading=217

    lat_name="lat"
    lon_name="lon"

#     print(file_list)
    for i in file_list:
#         print(i)
        data = Dataset(i[0], 'r')
        var = data[var_name][:,82:144,134:188]*86400
        lat = data[lat_name][:][82:144]
        lon = data[lon_name][:][134:188]
#         print(var.shape)
        data.close()
    #         lr=dpt.read_access_data(i,idx=time_leading).data[82:144,134:188]*86400
        result= torch.zeros((217,1,316,376),dtype=torch.float32)

        for idx,j in enumerate(var):
            if idx>= 217:
                break
            lr=dpt.interp_tensor_2d(j,(79,94))
            result[idx]=lr_transforms(Image.fromarray(lr))
            


        
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        net=torch.load('../save/model/vdsr_pr/best_test.pth',map_location=device)
        def prepare( l, device=False):
            def _prepare(tensor):
                tensor = tensor.float()
                return tensor.to(device)
            return [_prepare(_l) for _l in l]
        llrr=prepare([result],device)[0]
        with torch.set_grad_enabled(False):
            sr=net(llrr)
        sr_np=np.squeeze(sr.cpu().numpy())
        
        data_50=[]
        station_name=[]
        for station in consparam.station_50_index_for_size_of_hr_sr_station_code.keys():
            idx_i=consparam.station_50_index_for_size_of_hr_sr_station_code[station][0]
            idx_j=consparam.station_50_index_for_size_of_hr_sr_station_code[station][1]
            data_50.append(sr_np[:,idx_i,idx_j])
            station_name.append(station)
        data_50=np.array(data_50)
        print(data_50.shape)
        station_name=np.array(station_name)
        
        if not os.path.exists('../Data/'+var_name+'/daily_vdsrd_50station/'+i[1]):
            os.makedirs('../Data/'+var_name+'/daily_vdsrd_50station/'+i[1])
        
        f_w = nc.Dataset('../Data/'+var_name+'/daily_vdsrd_50station/'+i[1]+"/da_"+var_name+"_"+i[2].strftime("%Y%m%d")+"_"+i[1]+'.nc','w',format = 'NETCDF4')

        f_w.createDimension('time',time_leading)
        f_w.createDimension('station',50)

        f_w.createVariable('station',np.int32,('station'))
        f_w.createVariable('time',np.int,('time'))



        f_w.variables['station'][:] = station_name
        f_w.variables['time'][:] =  np.array(range(1,218))


        f_w.createVariable( 'pr', np.float32, ('station','time'))
        f_w.variables['pr'][:] = data_50

        f_w.close()
            

    
            

    
            
if __name__=='__main__':
    get_access_SR_data()

<class 'netCDF4._netCDF4.Variable'>
float32 pr(time, lat, lon)
    long_name: precipition
    cell_methods: time0: mean
    units: mm/day
    _FillValue: 1e+20
    missing_value: 1e+20
unlimited dimensions: time
current shape = (217, 691, 886)
filling on

In [ ]:
import sys
sys.path.append('../')
import util.PrepareData as pdata
import util.data_processing_tool as dpt

from datetime import date
from netCDF4 import Dataset
import numpy as np
import netCDF4 as nc
from datetime import timedelta, date, datetime
import platform 
import torch
from torchvision import datasets, models, transforms
import os
from PIL import Image
import util.constant_param as consparam
start_date=date(2012, 1, 2)
end_date=date(2012,12,25)
dates=[start_date + timedelta(x) for x in range((end_date - start_date).days + 1)]
from model import vdsrd2


def get_filename_with_no_time_order(rootdir):
    '''get filename first and generate label '''
    _files = []
    list = os.listdir(rootdir) #列出文件夹下所有的目录与文件
    for i in range(0,len(list)):
        path = os.path.join(rootdir,list[i],)
        if os.path.isdir(path):
            _files.extend(get_filename_with_no_time_order(path))
        if os.path.isfile(path):
            if path[-3:]==".nc":
                _files.append(path)
    return _files


def get_filename_with_time_order(rootdir,ensemble,dates,var_name):
    '''get filename first and generate label ,one different w'''
    _files = []
    for en in ensemble:
        for date in dates:
            access_path=rootdir+en+"/"+"da_"+var_name+"_"+date.strftime("%Y%m%d")+"_"+en+".nc"
#             print(access_path)
            if os.path.exists(access_path):
#                 print(access_path)
                path=[access_path]
                path.append(en)
                path.append(date)
                _files.append(path)

#最后去掉第一行，然后shuffle
#     if nine2nine and date_minus_one==1:
#         del _files[0]
    return _files

def to01(lr):
    return (lr-np.min(lr))/(np.max(lr)-np.min(lr))



#50 station
def get_access_SR_data():# 顺便保存 BI数据
    
    
    lr_transforms = transforms.Compose([
    transforms.Resize((316, 376)),
#     transforms.RandomResizedCrop(IMG_SIZE),
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomRotation(30),
    transforms.ToTensor()
#     transforms.Normalize(IMG_MEAN, IMG_STD)
    ])
    
    
    import netCDF4 as nc
#     ensemble=['e01','e02']
    sys = platform.system()
    init_date=date(1970, 1, 1)
    start_date=date(1990, 1, 1)
    end_date=date(2012,12,25) #if 929 is true we should substract 1 day  
    if sys == "Windows":
        init_date=date(1970, 1, 1)
        start_date=date(1990, 1, 1)
        end_date=date(2012,12,25) #if 929 is true we should substract 1 day   
        file_ACCESS_dir="H:/climate/access-s1/" 
        file_BARRA_dir="D:/dataset/accum_prcp/"
    #         args.file_ACCESS_dir="E:/climate/access-s1/"
    #         args.file_BARRA_dir="C:/Users/JIA059/barra/"
        file_DEM_dir="../DEM/"
    else:
        file_ACCESS_dir_pr="/g/data/ub7/access-s1/hc/raw_model/atmos/pr/daily/"
        file_ACCESS_dir="/g/data/ub7/access-s1/hc/raw_model/atmos/"
        # training_name="temp01"
        file_BARRA_dir="/g/data/ma05/BARRA_R/v1/forecast/spec/accum_prcp/"

        
    ensemble=['e01','e02','e03','e04','e05','e06','e07','e08','e09','e10','e11']

    var_name="pr"
    dates=[start_date + timedelta(x) for x in range((end_date - start_date).days + 1)]
    file_list=get_filename_with_time_order(file_ACCESS_dir+var_name+'/daily/',ensemble,dates,var_name)
    file_zg_list=get_filename_with_time_order(file_ACCESS_dir+var_name+'/daily/',ensemble,dates,'zg')

#             lr_zg=dpt.read_access_data(self.file_ACCESS_dir,en,access_date,time_leading,"zg")[-3,82:144,134:188]

    time_leading=217

    lat_name="lat"
    lon_name="lon"

    print(file_zg_list)
    for i,k in zip(file_list,file_zg_list):
#         print(i)
        data = Dataset(i[0], 'r')
        var = data[var_name][:,82:144,134:188]*86400
        
        zg_var = data['zg'][-3,82:144,134:188]
        lr_zg=to01(zg_var)
        lr_zg=lr_zg*np.max(var)

        lat = data[lat_name][:][82:144]
        lon = data[lon_name][:][134:188]
#         print(var.shape)
        data.close()
    #         lr=dpt.read_access_data(i,idx=time_leading).data[82:144,134:188]*86400
        result= torch.zeros((217,1,316,376),dtype=torch.float32)

        for idx,j in enumerate(var):
            if idx>= 217:
                break
            lr=dpt.interp_tensor_2d(j,(79,94))
            result[idx]=lr_transforms(Image.fromarray(lr))
            
        result_zg= torch.zeros((217,1,316,376),dtype=torch.float32)
        for idx,j in enumerate(var_zg):
            if idx>= 217:
                break
            result[idx]=lr_transforms(Image.fromarray(j))            


        
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        net_state=torch.load("../save/model/zg_val10/best_test_50.pth")['model'].module.state_dict()
        net = vdsr()
        net.load_state_dict(net_state)

#         net=torch.load('../save/model/vdsr_pr/best_test.pth',map_location=device)
        def prepare( l, device=False):
            def _prepare(tensor):
                tensor = tensor.float()
                return tensor.to(device)
            return [_prepare(_l) for _l in l]
        pr,zg=prepare([result,result_zg],device)


        with torch.set_grad_enabled(False):
            sr = net(pr,zg)
        sr_np=np.squeeze(sr.cpu().numpy())
        
        data_50=[]
        station_name=[]
        for station in consparam.station_50_index_for_size_of_hr_sr_station_code.keys():
            idx_i=consparam.station_50_index_for_size_of_hr_sr_station_code[station][0]
            idx_j=consparam.station_50_index_for_size_of_hr_sr_station_code[station][1]
            data_50.append(sr_np[:,idx_i,idx_j])
            station_name.append(station)
        data_50=np.array(data_50)
        print(data_50.shape)
        station_name=np.array(station_name)
        
        if not os.path.exists('../Data/'+var_name+'/daily_vdsrd2_50station/'+i[1]):
            os.makedirs('../Data/'+var_name+'/daily_vdsrd2_50station/'+i[1])
        
        f_w = nc.Dataset('../Data/'+var_name+'/daily_vdsrd2_50station/'+i[1]+"/da_"+var_name+"_"+i[2].strftime("%Y%m%d")+"_"+i[1]+'.nc','w',format = 'NETCDF4')

        f_w.createDimension('time',time_leading)
        f_w.createDimension('station',50)

        f_w.createVariable('station',np.int32,('station'))
        f_w.createVariable('time',np.int,('time'))



        f_w.variables['station'][:] = station_name
        f_w.variables['time'][:] =  np.array(range(1,218))


        f_w.createVariable( 'pr', np.float32, ('station','time'))
        f_w.variables['pr'][:] = data_50

        f_w.close()
            

            
if __name__=='__main__':
    get_access_SR_data()

In [1]:
import netCDF4 as nc

nc.Dataset('../../Data/da_zg_19900101_e01.nc')['zg'][:,-3,82:144,134:188].shape

E:\Users\Weifa\Anaconda3\envs\py37\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
E:\Users\Weifa\Anaconda3\envs\py37\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
E:\Users\Weifa\Anaconda3\envs\py37\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
E:\Users\Weifa\Anaconda3\envs\py37\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


(217, 62, 54)

In [2]:
nc.Dataset('../../Data/da_pr_19900109_e01.nc')['pr'][:,82:144,134:188].shape

(217, 62, 54)

In [3]:

import torch
from torchvision import datasets, models, transforms
import numpy as np
from model import vdsrd2
from PIL import Image

lr_transforms = transforms.Compose([
    transforms.Resize((316, 376)),
#     transforms.RandomResizedCrop(IMG_SIZE),
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomRotation(30),
    transforms.ToTensor()
#     transforms.Normalize(IMG_MEAN, IMG_STD)
])

lr=np.random.rand(79,94)
lr_transforms(Image.fromarray(lr))

result= torch.zeros((2,1,316,376),dtype=torch.float32)
result[0]=lr_transforms(Image.fromarray(lr))
# torch.tensor(sr).shape
# self.lr_transform(Image.fromarray(lr))
net_state=torch.load("../save/model/zg_val2012/best_test_20.pth")['model'].module.state_dict()
net = vdsrd2()
net.load_state_dict(net_state)

C:\Users\Weifa\AppData\Roaming\Python\Python37\site-packages\torchvision\transforms\functional.py:92: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:141.)
  img = torch.from_numpy(np.array(pic, np.float32, copy=False))
E:\Users\Weifa\Anaconda3\envs\py37\lib\site-packages\torch\serialization.py:658: SourceChangeWarning: source code of class 'torch.nn.parallel.data_parallel.DataParallel' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, Sour

<All keys matched successfully>

In [12]:
def prepare( l, device=False):
    def _prepare(tensor):
        tensor = tensor.float()
        return tensor.to(device)
    return [_prepare(_l) for _l in l]
llrr=prepare([result],device)[0]
llrr.shape

torch.Size([2, 1, 316, 376])

In [5]:
result.to(device)
with torch.set_grad_enabled(False):
    aaaa=net(result.to(device))

In [15]:
np.squeeze (aaaa.cpu().numpy() ).shape




(2, 316, 376)